In [ ]:
!pip install MLB-StatsAPI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 41.6 MB/s eta 0:00:00


In [ ]:
import requests
import datetime
import statsapi


In [ ]:
schedule = statsapi.schedule(sportId=1)
games = [game['game_id'] for game in schedule]
params = {
    "sportId": 1,
    "gamePk": 778253,
    "hydrate": "lineups",
}

gamedata = statsapi.get("schedule", params)
gamedata

{'copyright': 'Copyright 2025 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'totalItems': 1,
 'totalEvents': 0,
 'totalGames': 1,
 'totalGamesInProgress': 0,
 'dates': [{'date': '2025-04-19',
   'totalItems': 1,
   'totalEvents': 0,
   'totalGames': 1,
   'totalGamesInProgress': 0,
   'games': [{'gamePk': 778253,
     'gameGuid': '5d60c07e-6d59-4ca7-8949-2c7eaf574bc9',
     'link': '/api/v1.1/game/778253/feed/live',
     'gameType': 'R',
     'season': '2025',
     'gameDate': '2025-04-19T23:10:00Z',
     'officialDate': '2025-04-19',
     'status': {'abstractGameState': 'Final',
      'codedGameState': 'F',
      'detailedState': 'Final',
      'statusCode': 'F',
      'startTimeTBD': False,
      'abstractGameCode': 'F'},
     'teams': {'away': {'leagueRecord': {'wins': 10,
        'losses': 11,
        'pct': '.476'},
       'score': 3,
       'team': {'id': 133, 'name': 'Athle

In [ ]:
def batting_orders(game_json):
    batting_orders = {}
    boxscores = game_json["liveData"]["boxscore"]["teams"]
    home = boxscores["home"]["battingOrder"]
    away = boxscores["away"]["battingOrder"]
    batting_orders["home"] = home
    batting_orders["away"] = away

    return batting_orders


game = statsapi.get('game',{'gamePk': 778253})

batt = batting_orders(game)
batt

{'home': [668930,
  694192,
  592885,
  661388,
  686217,
  656555,
  669003,
  687401,
  702332],
 'away': [671732,
  667670,
  691016,
  669127,
  668709,
  805779,
  609280,
  570482,
  691777]}

In [ ]:
# https://github.com/jldbc/pybaseball/blob/master/docs/playerid_reverse_lookup.md

In [2]:
# --- Example Usage ---

import random


# Simulate results (replace with your actual all_results list)
all_results_example = []
for _ in range(10000): # Simulate 10k runs for example
     sim_run = {}
     for inn in [1,2,3]:
         sim_run[f'inning_{inn}'] = {
             'away': {'H': random.randint(0,6), 'R': 0, 'BB': random.randint(0,4), 'HR': random.randint(0,3)},
             'home': {'H': random.randint(0,6), 'R': 0, 'BB': random.randint(0,4), 'HR': random.randint(0,3)}
         }
         # Ensure HR <= H
         sim_run[f'inning_{inn}']['away']['HR'] = min(sim_run[f'inning_{inn}']['away']['HR'], sim_run[f'inning_{inn}']['away']['H'])
         sim_run[f'inning_{inn}']['home']['HR'] = min(sim_run[f'inning_{inn}']['home']['HR'], sim_run[f'inning_{inn}']['home']['H'])
     all_results_example.append(sim_run)


num_sims_example = len(all_results_example)

# Calculate probabilities and odds
df_final_probs = calculate_probabilities_and_odds(all_results_example, num_sims_example, max_bin_val=5)

if df_final_probs is not None:
    print("\n--- Final Probabilities and Odds DataFrame ---")
    with pl.Config(tbl_rows=50): # Show more rows
        print(df_final_probs)


--- Final Probabilities and Odds DataFrame ---
shape: (108, 7)
┌────────┬──────┬──────┬────────────┬─────────────┬──────────────┬───────────────┐
│ inning ┆ team ┆ stat ┆ number_bin ┆ probability ┆ decimal_odds ┆ american_odds │
│ ---    ┆ ---  ┆ ---  ┆ ---        ┆ ---         ┆ ---          ┆ ---           │
│ i64    ┆ cat  ┆ cat  ┆ str        ┆ f64         ┆ f64          ┆ i64           │
╞════════╪══════╪══════╪════════════╪═════════════╪══════════════╪═══════════════╡
│ 1      ┆ away ┆ H    ┆ 0          ┆ 0.1394      ┆ 7.173601     ┆ 617           │
│ 1      ┆ away ┆ H    ┆ 1          ┆ 0.1516      ┆ 6.596306     ┆ 560           │
│ 1      ┆ away ┆ H    ┆ 2          ┆ 0.1362      ┆ 7.342144     ┆ 634           │
│ 1      ┆ away ┆ H    ┆ 3          ┆ 0.1395      ┆ 7.168459     ┆ 617           │
│ 1      ┆ away ┆ H    ┆ 4          ┆ 0.1422      ┆ 7.032349     ┆ 603           │
│ 1      ┆ away ┆ H    ┆ 5+         ┆ 0.2911      ┆ 3.435246     ┆ 244           │
│ 1      ┆ away ┆ BB   

In [ ]:
import polars as pl
import collections
import math
import numpy as np # Used for isclose, infinity
import random # For example data generation

# --- Helper Functions for Odds Conversion ---

def probability_to_decimal_odds(probability: float) -> float | None:
    """Converts probability to decimal odds."""
    if probability is None or probability <= 0:
        return None # Or np.inf, or a very large number like 9999
    # Avoid division by very small number leading to massive odds if desired
    # if probability < 1e-9: return 9999.0
    try:
        return 1.0 / probability
    except ZeroDivisionError:
        return None # Or np.inf

def probability_to_american_odds(probability: float) -> int | None:
    """Converts probability to American odds."""
    if probability is None or probability <= 0 or probability >= 1:
         # Handle edge cases: P=0 -> infinite odds, P=1 -> infinitely favored
         # Representing these perfectly is hard, return None or placeholder
         if probability == 1: return -99999 # Example for P=1
         return None # For P=0 or invalid

    if probability < 1e-9: return 99999 # Avoid massive odds for tiny probs

    if probability < 0.5:
        # Underdog calculation: +odds
        odds = (100 / probability) - 100
    else:
        # Favorite calculation: -odds
        odds = -1 * (probability / (1 - probability)) * 100

    # Round to nearest integer
    return int(round(odds))

# --- Main Analysis Function ---

def calculate_probabilities_and_odds(all_results: list, num_simulations: int, max_bin_val: int = 5):
    """
    Analyzes simulation results to calculate probabilities and odds for
    Hits (H), Walks (BB), and Home Runs (HR) per team per inning (0-5+).

    Args:
        all_results (list): List of dictionaries, each representing one simulation run.
                            Example element: {'inning_1': {'away': {'H':h,'R':r,'BB':bb,'HR':hr}, ...}}
        num_simulations (int): The total number of simulations run.
        max_bin_val (int): The threshold for the final '+' bin (e.g., 5 means bins 0,1,2,3,4,5+).

    Returns:
        polars.DataFrame: A DataFrame with columns: inning, team, stat, number_bin,
                          probability, decimal_odds, american_odds.
                          Returns None if num_simulations is zero or input is empty.
    """
    if not all_results or num_simulations <= 0:
        print("Warning: No simulation results provided or num_simulations is zero. Cannot calculate probabilities.")
        return None

    bins = list(range(max_bin_val)) # e.g., [0, 1, 2, 3, 4]
    plus_bin_label = f"{max_bin_val}+" # e.g., "5+"

    # --- Initialize dictionaries to store counts ---
    results_counts = {}
    for inn in [1, 2, 3]:
        results_counts[f'inning_{inn}'] = {
            'away': {'H': collections.defaultdict(int), 'BB': collections.defaultdict(int), 'HR': collections.defaultdict(int)},
            'home': {'H': collections.defaultdict(int), 'BB': collections.defaultdict(int), 'HR': collections.defaultdict(int)}
        }

    # --- Tally results into bins ---
    for sim_result in all_results:
        for inn in [1, 2, 3]:
            inn_key = f'inning_{inn}'
            if inn_key not in sim_result: continue # Skip if inning missing

            for team in ['away', 'home']:
                if team not in sim_result[inn_key]: continue # Skip if team missing

                try:
                    team_result = sim_result[inn_key][team]
                    h = team_result.get('H', 0) # Use .get() for safety
                    bb = team_result.get('BB', 0)
                    hr = team_result.get('HR', 0)

                    # Determine bin (0 to max_bin_val)
                    h_bin = min(h, max_bin_val)
                    bb_bin = min(bb, max_bin_val)
                    hr_bin = min(hr, max_bin_val)

                    # Increment count for the corresponding bin
                    results_counts[inn_key][team]['H'][h_bin] += 1
                    results_counts[inn_key][team]['BB'][bb_bin] += 1
                    results_counts[inn_key][team]['HR'][hr_bin] += 1
                except TypeError:
                     # Handle cases where H/BB/HR might not be numbers if sim failed partially
                     print(f"Warning: Invalid data type found for {inn_key}/{team} in a sim result. Skipping entry.")
                     continue
                except KeyError as e:
                    print(f"Warning: Missing key {e} in {inn_key}/{team}. Skipping entry.")
                    continue # Skip this entry if structure is wrong

    # --- Calculate probabilities and odds, structure for DataFrame ---
    data_for_df = []
    for inn in [1, 2, 3]:
        inn_key = f'inning_{inn}'
        if inn_key not in results_counts: continue

        for team in ['away', 'home']:
            if team not in results_counts[inn_key]: continue

            for stat in ['H', 'BB', 'HR']:
                if stat not in results_counts[inn_key][team]: continue

                stat_counts = results_counts[inn_key][team][stat]
                total_for_plus_bin = 0 # Keep track for verification if needed

                # Calculate probabilities for bins 0 to max_bin_val
                for i in range(max_bin_val + 1): # Iterate 0, 1, 2, 3, 4, 5
                    count = stat_counts.get(i, 0)
                    probability = count / num_simulations
                    decimal_odds = probability_to_decimal_odds(probability)
                    american_odds = probability_to_american_odds(probability)

                    # Determine bin label ('0', '1', ..., '4', '5+')
                    number_bin_label = f"{i}" if i < max_bin_val else plus_bin_label

                    data_for_df.append({
                        "inning": inn,
                        "team": team,
                        "stat": stat,
                        "number_bin": number_bin_label,
                        "probability": probability,
                        "american_odds": american_odds,
                        "decimal_odds": decimal_odds,
                    })
                    # Break after processing the max_bin_val bin (which represents X+)
                    if i == max_bin_val:
                         break


    # --- Create Polars DataFrame ---
    if not data_for_df:
        print("Warning: No data processed for DataFrame creation.")
        return pl.DataFrame() # Return empty DataFrame

    schema = {
        "inning": pl.Int64,
        "team": pl.Categorical,
        "stat": pl.Categorical,
        "number_bin": pl.Utf8, # e.g., '0', '1', '5+'
        "probability": pl.Float64,
        "decimal_odds": pl.Float64, # Can be null
        "american_odds": pl.Int64    # Can be null
    }

    df_probabilities = pl.DataFrame(data_for_df, schema=schema)

    return df_probabilities.sort(["inning", "team", "stat", "number_bin"])